In [1]:
import scanpy as sc
import squidpy as sq
import numpy as np
import pandas as pd
import anndata as ad
from anndata import AnnData
import pathlib
import matplotlib.pyplot as plt
import matplotlib as mpl
import skimage
import os
import time

In [2]:
# import tangram for spatial deconvolution
import tangram as tg

In [27]:
stdir = "/home/comp/cszrwang/data/osmfish/"
stfile = 'osmfish.st.cnt.genexrow.tsv'
cellfile = "osmfish.cell_proportion.txt"

scdir = "/home/comp/cszrwang/data/osmfish/SSp_ref/external/"
reffile = "sc_cnt.33gene.5392cell.genexrow.tsv"
metafile = "sc_mta.5392cell.tsv"

resultdir = "./Tangram/"
resultfile = "osmfish.tangram.csv"
result_cellfile = "osmfish.tangram.csv"
if not os.path.exists(resultdir):
    os.mkdir(resultdir)

In [4]:
# read in ST data
st = pd.read_csv(stdir + stfile, sep='\t', index_col=0)
st = st.transpose()
adata_st = AnnData(st)
adata_st

AnnData object with n_obs × n_vars = 737 × 33

In [5]:
## append cell count for each spot
cell = pd.read_csv(stdir + cellfile, sep='\t', index_col=0)
adata_st.obs = adata_st.obs.merge(cell.sum(axis = 1).to_frame(name="cell_count"), how = 'outer', left_index = True, right_index = True)

## create spatial coordinates information
spatial_coord = adata_st.obs.reset_index()['index'].str.split('_', expand = True).to_numpy().astype(int)
spatial_coord[:,0] = spatial_coord[:,0] + spatial_coord[:,1]
spatial_coord = spatial_coord[:, 0:2]
adata_st.obsm['spatial'] = spatial_coord

centroid = pd.Series(index = adata_st.obs.index, dtype = "object")
for i in range(len(centroid)):
    centroid[i] = np.tile(spatial_coord[i], (adata_st.obs.cell_count[i],1))

adata_st.obsm['image_features'] = cell.sum(axis = 1).to_frame(name="segmentation_label").merge(centroid.to_frame(name = "segmentation_centroid"),left_index = True, right_index = True)

In [6]:
spatial_coord

array([[28000, 27600],
       [28800, 28400],
       [29600, 29200],
       ...,
       [58400, 35600],
       [60000, 37200],
       [60800, 37200]])

In [21]:
# Read in scRNA-seq data
scdat = pd.read_csv(scdir + reffile, sep='\t', index_col=0)
adata_sc = AnnData(scdat.T)

sc_meta = pd.read_csv(scdir + metafile, sep='\t')
sc_meta.set_index('sample_name', inplace = True)
sc_meta.index = sc_meta.index.astype('str')
adata_sc.obs = adata_sc.obs.merge(sc_meta, how = 'left', left_index=True, right_index=True)
adata_sc.obs["bio_celltype"] = pd.Categorical(adata_sc.obs['bio_celltype'])
adata_sc

start_time = time.time()
# preprocessing: find the common genes between sc and st
tg.pp_adatas(adata_sc, adata_st)

INFO:root:33 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:33 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.


In [32]:
# 用了额外的信息！！！！
# Deconvolution
ad_map = tg.map_cells_to_space(
    adata_sc,
    adata_st,
    mode="constrained",
    target_count=adata_st.obs.cell_count.sum(),
    density_prior=np.array(adata_st.obs.cell_count) / adata_st.obs.cell_count.sum(),
    num_epochs=1000,
    device="cuda:0",
    #device='cpu',
)

INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 33 genes and customized density_prior in constrained mode...


Score: 0.619, KL reg: 0.163, Count reg: 1139.114, Lambda f reg: 1109.579
Score: 0.995, KL reg: 0.000, Count reg: 0.002, Lambda f reg: 173.983
Score: 0.998, KL reg: 0.000, Count reg: 0.072, Lambda f reg: 51.820
Score: 0.998, KL reg: 0.000, Count reg: 0.041, Lambda f reg: 34.808
Score: 0.998, KL reg: 0.000, Count reg: 0.142, Lambda f reg: 29.927
Score: 0.999, KL reg: 0.000, Count reg: 0.078, Lambda f reg: 23.930
Score: 0.999, KL reg: 0.000, Count reg: 0.158, Lambda f reg: 21.380
Score: 0.999, KL reg: 0.000, Count reg: 0.117, Lambda f reg: 18.938
Score: 0.999, KL reg: 0.000, Count reg: 0.228, Lambda f reg: 18.325
Score: 0.999, KL reg: 0.000, Count reg: 0.135, Lambda f reg: 15.487


INFO:root:Saving results..


In [33]:
# Gather deconvolution results
## map the cell type information to the st AnnData object
## The output created is the unnormalized probability matrix
tg.project_cell_annotations(ad_map, adata_st, annotation="bio_celltype")
end_time = time.time()
print("--- %.2f seconds ---" % (time.time() - start_time))

INFO:root:spatial prediction dataframe is saved in `obsm` `tangram_ct_pred` of the spatial AnnData.


--- 1581.73 seconds ---


In [34]:
## normalize the probability matrix and save as csv
prob_mat = adata_st.obsm["tangram_ct_pred"]
prob_mat = prob_mat.div(prob_mat.sum(axis=1), axis=0)
prob_mat.to_csv(resultdir + resultfile, sep = '\t')


## create cell-level mapping assignments
tg.create_segment_cell_df(adata_st)
tg.count_cell_annotations(
    ad_map,
    adata_sc,
    adata_st,
    annotation="bio_celltype",
)
adata_st.obsm["tangram_ct_count"].drop(columns = ['centroids']).to_csv(resultdir + result_cellfile)

INFO:root:cell segmentation dataframe is saved in `uns` `tangram_cell_segmentation` of the spatial AnnData.
INFO:root:spot centroids is saved in `obsm` `tangram_spot_centroids` of the spatial AnnData.
INFO:root:spatial cell count dataframe is saved in `obsm` `tangram_ct_count` of the spatial AnnData.
